## 栅格转矢量

In [1]:
from osgeo import gdal, ogr
import matplotlib.pyplot as plt

In [ ]:
path_mndwi = "E:/YNU/5/OpenSourceGIS/Act_5/OnClass/呈贡10m分辨率裁剪子区域及呈贡20m分辨率/chenggong_utm48_20m_mndwi.tif"
dset = gdal.Open(path_mndwi)
mndwi = dset.GetRasterBand(1)
mndwi_array = mndwi.ReadAsArray()
print(mndwi_array.shape)

In [ ]:
plt.imshow(mndwi_array)

In [ ]:
path_mndwi2polygon = 'E:/YNU/5/OpenSourceGIS/Act_5/OnClass/矢量转栅格/chenggong_utm48_20m_mndwi_thre01_vec.gpkg'

proj = gdal.osr.SpatialReference()
proj.ImportFromWkt(dset.GetProjection())

driver_vec = ogr.GetDriverByName('GPKG')
ds_vec = driver_vec.CreateDataSource(path_mndwi2polygon)
poly_layer = ds_vec.CreateLayer('mndwi_polygon', srs=proj, geom_type=ogr.wkbMultiPolygon)
Field_DN = ogr.FieldDefn('DN', ogr.OFTInter)
poly_layer.CreateField(Field_DN)
gdal.Polygonize(srcBand=mndwi, maskBand=None, outLayer=poly_layer, iPixValField=0)

DN_vec = [1]
for fea in poly_layer:
    DN_value = fea.GetField('DN')
    if DN_value not in DN_vec:
        poly_layer.DeleteFeature(fea.GetFiD())

ds_vec = None
dset = None

## 矢量转栅格

In [ ]:
path_vec ='E:/YNU/5/OpenSourceGIS/Act_5/OnClass/栅格转矢量/chenggong_utm48_20m_mndwi_thre01_vec.gpkg'
path_raster='E:/YNU/5/OpenSourceGIS/Act_5/OnClass/矢量转栅格/chenggong_utm48_20m_mndwi_thre01_vecZraster.tif'

In [ ]:
ds_vec = ogr.Open(path_vec)
layer_vec = ds_vec.GetLayer()
proj_vec = layer_vec.GetSpatialRef()
proj_vec_wkt = proj_vec.ExportTowkt()
print(proj_vec_wkt)

x_min, x_max,y_min, y_max = layer_vec.GetExtent()
print(x_min, x_max, y_min, y_max)

In [ ]:
pixel_size = 20
geo_trans =[x_min, 20,0, y_max, 0,-20]
x_size = int((x_max-x_min)/pixel_size)
y_size = int((y_max-y_min)/pixel_size)

driver = gdal.GetDriverByName('GTiff')
dset_raster = driver.Create(path_raster,
                            xsize=x_size,
                            ysize=y_size,
                            bands=1,
                            eType=gdal.GDT_Int16
                            )

In [ ]:
gdal.RasterizeLayer(dataset=dset_raster, bands=[1], layer=layer_vec, options=['ATTRIBUTE=DN'])